In [30]:
# TEST CODE
import os
import time

import nltk
import wrds
import pandas as pd

In [31]:
# TEST CODE
conn = wrds.Connection(wrds_username='denisfench')

Loading library list...
Done


In [32]:
# TEST CODE
### Get S&P500 Index Membership from CRSP

sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])


### Add Other Company Identifiers from CRSP.MSENAMES
### - You don't need this step if only PERMNO is required
### - This step aims to add TICKER, SHRCD, EXCHCD and etc.

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt,
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

print("The length of the full SP500 list is: ", len(sp500_full))

The length of the full SP500 list is:  985474


In [33]:
# TEST CODE
# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]


### Add Other Company Identifiers from CRSP.MSENAMES

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt,
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]


### Add Compustat Identifiers

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno,
                  linktype, linkprim, linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]

# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', 'linktype', \
                                  'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip',\
                     'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]


### Add CIKs and Link with SEC Index Files using CIK

names = conn.raw_sql(""" select gvkey, cik, sic, naics, gind, gsubind from comp.names """)

# Merge sp500 constituents table with names table
sp500 = pd.merge(sp500ccm, names, on='gvkey',  how='left')
sp500.head(20)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret,cik,sic,naics,gind,gsubind
0,2000-01-31,40416.0,AVON PRODUCTS INC,05430310,11.0,1.0,2844.0,AVP,001920,01,1967-05-18,2015-03-20,-0.035985,0000008868,2844,325620,303020,30302010
1,2000-01-31,44062.0,SPRINGS INDUSTRIES INC,85178310,11.0,1.0,2221.0,SMI,009963,01,1967-06-29,2000-12-11,-0.089202,0000093102,2211,313210,252010,25201020
2,2000-01-31,26403.0,DISNEY WALT CO,25468710,11.0,1.0,4833.0,DIS,003980,01,1976-07-01,2022-03-31,0.241453,0001744489,4888,515120,502020,50202010
3,2000-01-31,60628.0,FEDEX CORP,31428X10,11.0,1.0,4513.0,FDX,004598,01,1980-11-06,2022-03-31,-0.033588,0001048911,4513,492110,203010,20301010
4,2000-01-31,69032.0,MORGAN STANLEY DEAN WITTER & CO,61744644,11.0,1.0,6282.0,MWD,012124,01,1995-09-22,2022-03-31,-0.069002,0000895421,6211,523110,402030,40203020
5,2000-01-31,21186.0,WESTVACO CORP,96154810,11.0,1.0,2631.0,W,011446,01,1957-03-01,2022-03-31,-0.159004,0001159297,2631,322130,151030,15103020
6,2000-01-31,52978.0,HASBRO INC,41805610,11.0,1.0,3944.0,HAS,005518,01,1984-09-13,2022-03-31,-0.204752,0000046080,3944,339930,252020,25202010
7,2000-01-31,21371.0,CARDINAL HEALTH INC,14149Y10,11.0,1.0,5122.0,CAH,002751,01,1997-05-27,2022-03-31,-0.003916,0000721371,5122,424210,351020,35102010
8,2000-01-31,75333.0,BURLINGTON RESOURCES INC,12201410,11.0,1.0,1311.0,BR,015084,01,1993-11-23,2006-03-31,-0.030246,0000833320,1311,211111,101020,10102020
9,2000-01-31,23317.0,ENRON CORP,29356110,11.0,1.0,1311.0,ENE,006127,01,1951-09-06,2001-11-29,0.529577,0001024401,5172,422720,551050,55105010


In [78]:
import wrds
import pandas as pd

# this function retrieves all sp500 records from the WRDS database
# the records are returned in Pandas dataframe format
def get_sp500_records():
    # establish connection to the WRDS database
    conn = wrds.Connection(wrds_username='denisfench')

    ### Get S&P500 Index Membership from CRSP
    sp500 = conn.raw_sql("""
                            select a.*, b.date, b.ret
                            from crsp.msp500list as a,
                            crsp.msf as b
                            where a.permno=b.permno
                            and b.date >= a.start and b.date<= a.ending
                            and b.date>='01/01/2000'
                            order by date;
                            """, date_cols=['start', 'ending', 'date'])

    ### Add Other Company Identifiers from CRSP.MSENAMES, such as TICKER, SHRCD, EXCHCD
    mse = conn.raw_sql("""
                            select comnam, ncusip, namedt, nameendt,
                            permno, shrcd, exchcd, hsiccd, ticker
                            from crsp.msenames
                            """, date_cols=['namedt', 'nameendt'])

    # if nameendt is missing then set to today date
    mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

    # Merge with SP500 data
    sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

    # Impose the date range restrictions
    sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                                & (sp500_full.date<=sp500_full.nameendt)]

    ### Add Other Company Identifiers from CRSP.MSENAMES
    mse = conn.raw_sql("""
                            select comnam, ncusip, namedt, nameendt,
                            permno, shrcd, exchcd, hsiccd, ticker
                            from crsp.msenames
                            """, date_cols=['namedt', 'nameendt'])

    # if nameendt is missing then set to today date
    mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

    # Merge with SP500 data
    sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

    # Impose the date range restrictions
    sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                                & (sp500_full.date<=sp500_full.nameendt)]

    ### Add Compustat Identifiers
    ccm=conn.raw_sql("""
                      select gvkey, liid as iid, lpermno as permno,
                      linktype, linkprim, linkdt, linkenddt
                      from crsp.ccmxpf_linktable
                      where substr(linktype,1,1)='L'
                      and (linkprim ='C' or linkprim='P')
                      """, date_cols=['linkdt', 'linkenddt'])

    # if linkenddt is missing then set to today date
    ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

    # Merge the CCM data with S&P500 data
    # First just link by matching PERMNO
    sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

    # Then set link date bounds
    sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                            &(sp500ccm['date']<=sp500ccm['linkenddt'])]

    # Rearrange columns for final output
    sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', 'linktype', \
                                      'linkprim', 'linkdt', 'linkenddt'])
    sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip',\
                         'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                         'gvkey', 'iid', 'start', 'ending', 'ret']]

    ### Add CIKs and Link with SEC Index Files using CIK
    names = conn.raw_sql(""" select gvkey, cik, sic, naics, gind, gsubind from comp.names """)

    # Merge sp500 constituents table with names table
    sp500 = pd.merge(sp500ccm, names, on='gvkey',  how='left')
    return sp500

In [34]:
# TEST CODE
# Getting a two-year sample of records for 2020 and 2021

sp500_sample = sp500.loc['01/01/2020' <= sp500.date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

sp500_sample = sp500_sample.loc[sp500.date <= '01/01/2022'][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

In [35]:
# TEST CODE
sp500_sample

,date,permno,comnam,ncusip,gvkey,iid,cik,ticker,sic,naics
120131,2020-01-31,34746.0,FIFTH THIRD BANCORP,31677310,004640,01,0000035527,FITB,6020,522110
120132,2020-01-31,75591.0,IDEX CORP,45167R10,015267,01,0000832101,IEX,3561,333914
120133,2020-01-31,14541.0,CHEVRON CORP NEW,16676410,002991,01,0000093410,CVX,2911,324110
120134,2020-01-31,14593.0,APPLE INC,03783310,001690,01,0000320193,AAPL,3663,334220
120135,2020-01-31,75341.0,DUKE REALTY CORP,26441150,013510,01,0000783280,DRE,6798,531120
...,...,...,...,...,...,...,...,...,...,...
132138,2021-12-31,93096.0,DOLLAR GENERAL CORP NEW,25667710,004016,02,0000029534,DG,5331,452319
132139,2021-12-31,19502.0,WALGREENS BOOTS ALLIANCE INC,93142710,011264,01,0001618921,WBA,5912,446110
132140,2021-12-31,69796.0,CONSTELLATION BRANDS INC,21036P10,002710,02,0000016918,STZ,2082,312120
132141,2021-12-31,62092.0,THERMO FISHER SCIENTIFIC INC,88355610,010530,01,0000097745,TMO,3826,334516


In [81]:
# this function retrieves a subsample of the sp500 records corresponding to
# the given dates (YYYY-MM-DD)
# EX: get_sp500_records_by_date(sp500, '2020-01-01', '2021-12-31')

def get_sp500_records_by_date(sp500_record, start_date, end_date):
    new_sp500_sample = sp500_record.loc[start_date <= sp500_record
        .date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

    new_sp500_sample = new_sp500_sample.loc[sp500_record.date <=
                                      end_date][['date',
                                                                        'permno',
                                                          'comnam',
                                                   'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

    return new_sp500_sample

In [7]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# increment the current date by 1 month
def get_next_month(date):
    DATE_FORMAT = "%Y-%m-%d"
    datetime_date = datetime.strptime(date, DATE_FORMAT)
    new_date = datetime_date + relativedelta(months=1)
    return new_date.strftime(DATE_FORMAT)


def get_next_day(date):
    DATE_FORMAT = "%Y-%m-%d"
    datetime_date = datetime.strptime(date, DATE_FORMAT)
    new_date = datetime_date + relativedelta(day=1)
    return new_date.strftime(DATE_FORMAT)

In [8]:
# TODO: transition the granularity to day by day instad of month by month and
#  store the day the record has been retrieved

from sec_edgar_downloader import Downloader

def get_filing_by_month(ticker, date, dir_name):
    num_filings = 0
    dl = Downloader(dir_name + "/" + date)
    num_filings += dl.get("10-Q", ticker, after=date, before=get_next_month
    (date))
    num_filings += dl.get("10-K", ticker, after=date, before=get_next_month
    (date))
    return num_filings

def get_filing_by_day(ticker, date, dir_name):
    num_filings = 0
    dl = Downloader(dir_name + "/" + date)
    num_filings += dl.get("10-Q", ticker, after=date, before=get_next_day
    (date))
    num_filings += dl.get("10-K", ticker, after=date, before=get_next_day
    (date))
    return num_filings

In [9]:
# this function retrieves all S&P 500 10-Q or 10-K filings for a given month,
# if such a filing exists for a given company and writes them to a given directory
# the date should be passed in the following format: YYYY-MM-DD
def get_all_sp_filings_by_month(date, dir_name):
    num_filings = 0
    sp500_companies = sp500_sample.loc[sp500_sample.date == date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]
    sp500_tickers = sp500_companies['ticker']

    for ticker in sp500_tickers:
        num_filings += get_filing_by_month(ticker, date, dir_name)

    return num_filings

# this function retrieves all S&P 500 10-Q or 10-K filings for a given day,
# if such a filing exists for a given company and writes them to a given directory
# the date should be passed in the following format: YYYY-MM-DD
def get_all_sp_filings_by_day(date, dir_name):
    num_filings = 0
    sp500_companies = sp500_sample.loc[sp500_sample.date == date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]
    sp500_tickers = sp500_companies['ticker']

    for ticker in sp500_tickers:
        num_filings += get_filing_by_day(ticker, date, dir_name)

    return num_filings

In [ ]:
# TEST CODE

DATA_FOLDER = "data"

filings = get_all_sp_filings_by_month("2020-01-31", DATA_FOLDER)
filings

KeyboardInterrupt: 

In [ ]:
# TEST CODE
apple_filing = get_filing_by_day()

In [10]:
# TEST CODE
# parsing the records
from bs4 import BeautifulSoup

In [11]:
# TEST CODE
with open("data/sec-edgar-filings/AAPL/10-Q/0000320193-20-000010/filing-details.html") as fp:
    soup = BeautifulSoup(fp, "html.parser")

In [12]:
from bs4 import BeautifulSoup
import os

# this function retrieves all tickers for all companies from a given directory
def get_all_filing_tickers(dir_name):
    files = os.listdir(dir_name + "/sec-edgar-filings")
    return files

In [13]:
# TEST CODE
print(get_all_filing_tickers("data"))

['VZ', 'AMZN', 'CNP', 'RCL', 'CAT', 'AAPL', 'KHC', 'AGN', 'ANET', 'CAH', 'FBHS', 'PFE', 'REG', 'APTV', 'AAL', 'CDW', 'MAR', 'VRSN', 'KMI', 'SPGI', 'ALLE', 'SYK', 'PEP', 'FRT', 'SNPS', 'PLD', 'MMM', 'EMN', 'AMT', 'ADI', 'MAA', 'DRE', 'FTV', 'MGM', 'VLO', 'EMR', 'SYY', 'GILD', 'SNA', 'MS', 'ALXN', 'CVX', 'NCLH', 'UHS', 'MO', 'TXN', 'UNM', 'BK', 'BLL', 'TTWO', 'HCA', 'CDNS', 'HII', 'CTSH', 'MCHP', 'VTR', 'HIG', 'ETFC', 'IPG', 'DISH', 'UNP', 'AMAT', 'LLY', 'NTAP', 'WAB', 'REGN', 'RSG', 'IR', 'IQV', 'LMT', 'CI', 'PPL', 'ANSS', 'EL', 'PSX', 'JNJ', 'QCOM', 'LNT', 'BAC', 'IT', 'SIVB', 'DHI', 'IRM', 'DGX', 'EQR', 'ED', 'DHR', 'HSIC', 'ALB', 'NLOK', 'WM', 'COG', 'RHI', 'TMO', 'ALL', 'BSX', 'FAST', 'ABBV', 'AFL', 'RMD', 'TWTR', 'EFX', 'LYB', 'PNR', 'KEY', 'ATVI', 'CHRW', 'COF', 'PRGO', 'VMC', 'HSY', 'T', 'CHTR', 'TAP', 'MCK', 'AIG', 'EBAY', 'F', 'NTRS', 'O', 'CFG', 'STE', 'PYPL', 'IBM', 'AMD', 'BWA', 'MHK', 'PG', 'INCY', 'XOM', 'USB', 'SO', 'LRCX', 'FOXA', 'DTE', 'BRK', 'ROP', 'AME', 'RJF', 'KMB'

In [73]:
from datetime import datetime

def parse_date(filed_as_of_date_line):
    DATE_FORMAT = "%Y%m%d"
    date = ""
    for ch in filed_as_of_date_line:
        if ch.isdigit():
            date += ch
    return datetime.strptime(date, DATE_FORMAT).date()

In [74]:
# TEST CODE

parse_date("FILED AS OF DATE:		20200129")

datetime.date(2020, 1, 29)

In [75]:
from bs4 import BeautifulSoup

FILING_DETAILS = "filing-details.html"
FULL_SUBMISSION = "full-submission.txt"
FILED_AS_OF_DATE_IDX = 7

# this function will search for a filing in a given directory based on the
# given ticker
# it will return a dictionary with a ticker and the date of filing as the key
# and filing text as the value
# currently the function doesn't indicate whether the filing is a 10-Q or 10-K
def get_filings_from_ticker(ticker, dir_name, sec_filings):
    for root, dirs, files in os.walk(dir_name + "/sec-edgar-filings/" + ticker):
        for f in files:
            if FILING_DETAILS in f:
                with open(root + "/" + f) as fp:
                    soup = BeautifulSoup(fp, "html.parser")
                    line_idx = 0
                    with open(root + "/" + FULL_SUBMISSION) as fs_fp:
                        while line_idx < FILED_AS_OF_DATE_IDX:
                            line_idx += 1
                            fs_fp.readline()
                        date = parse_date(fs_fp.readline())
                    sec_filings[(ticker, date)] = soup.get_text()
    return sec_filings

In [77]:
# TEST CODE
filings = {}
filings = get_filings_from_ticker("AAL", "data", filings)
print(filings)

{('AAL', datetime.date(2020, 2, 19)): "\n\n\nDocument\n\n\n1.61.61.61.61.61.61.61.60.6250.6250.6250.6250.6250.6250.6250.625P1YP1YP1YP1Yfalse--12-31FY20190000006201000000451541000000410000000.10.10.10.10.10.10.10.10.10.10.10.0110.011175000000010001750000000100010001000460610870100042820250610000.07310.08390.02990.030.050.07310.080.08390.02990.050.030.0550.03450.04050.03450.04050.300.050.150.100.450.10.050.550.800.250.400.100.0500.0200.15000.200.450.100.150.020.300.050.150.100.450.10.200.550.800.250.400.10.0500.0200.15000.200.450.100.150.02P25YP25Y0.03540.03720.03800.03740.03540.03720.03800.03741000000000010000000000P30YP10YP30YP10YP20YP5YP5YP3YP30YP10YP30YP10YP20YP5YP5YP3Y24000000002400000000216686117091401682202339531271660615734090566\n\n0000006201\n\n\n2019-01-01\n2019-12-31\n\n\n\n0000006201\n\nsrt:SubsidiariesMember\n\n\n\n2019-01-01\n2019-12-31\n\n\n\n0000006201\n\n\n2020-02-14\n\n\n\n0000006201\n\n\n2019-06-30\n\n\n\n0000006201\n\nsrt:SubsidiariesMember\n\n\n\n2020-02-14\n\n\n\n0

In [7]:
# TEST CODE
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dennisfenchenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import ssl

# this function sets up SSL context for downloading packages from nltk library
def create_ssl_context_for_nltk():
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# a function that tokenizes 10-K or 10-Q corpus that
def tokenize_filing(filing_corpus):
    filing_tokenizer = RegexpTokenizer(r'\w+')
    return filing_tokenizer.tokenize(filing_corpus)

In [10]:
# a function to remove the stop words from the filings corpus
def filter_stopwords(tokenized_filing_corpus):
    stop_words = set(stopwords.words('english'))
    filing_corpus_filtered = [word for word in tokenized_filing_corpus if not \
        word.lower() in stop_words]
    return filing_corpus_filtered

def filter_numbers(filing_corpus):
    return [token for token in filing_corpus if not (token.isdigit()
                                         or token[0] == '-' and token[1:].isdigit())]

In [22]:
# TEST CODE
tokenized_aal_filing = tokenize_filing(filings.get('AAL'))
filtered_corpus = filter_stopwords(tokenized_aal_filing)
filtered_corpus = filter_numbers(filtered_corpus)
print("The size before the filtering " + str(len(tokenized_aal_filing)))
print("The size after the filtering " + str(len(filtered_corpus)))

The size before the filtering 118317
The size after the filtering 67698


In [109]:
# TEST CODE
# get CRSP stock performance for each given company in a given quarter
# we will use the Value-Weighted Return (including distributions) for that
conn.list_tables(library='crsp')

['acti',
 'asia',
 'asib',
 'asic',
 'asio',
 'asix',
 'bmdebt',
 'bmheader',
 'bmpaymts',
 'bmquotes',
 'bmyield',
 'bndprt06',
 'bndprt12',
 'bxcalind',
 'bxdlyind',
 'bxmthind',
 'bxquotes',
 'bxyield',
 'cap',
 'ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'contact_info',
 'crsp_cik_map',
 'crsp_daily_data',
 'crsp_header',
 'crsp_monthly_data',
 'crsp_names',
 'crsp_portno_map',
 'crsp_ziman_daily_index',
 'crsp_ziman_monthly_index',
 'cs20yr',
 'cs5yr',
 'cs90d',
 'cst_hist',
 'daily_nav',
 'daily_nav_ret',
 'daily_returns',
 'dividends',
 'dport1',
 'dport2',
 'dport3',
 'dport4',
 'dport5',
 'dport6',
 'dport7',
 'dport8',
 'dport9',
 'dsbc',
 'dsbo',
 'dse',
 'dse62',
 'dse62delist',
 'dse62dist',
 'dse62exchdates',
 'dse62names',
 'dse62nasdin',
 'dse62shares',
 'dseall',
 'dseall62',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',


In [115]:
# TEST CODE

company = conn.get_table(library='crsp', table='msp500p', obs=5)

In [116]:
# TEST CODE
company

,caldt,vwretd,vwretx,ewretd,ewretx,totval,totcnt,usdval,usdcnt,spindx,sprtrn
0,1925-12-31,NaN,NaN,NaN,NaN,15236829.5,89.0,NaN,NaN,12.46,NaN
1,1926-01-30,-0.001783,-0.003980,0.006457,0.003250,15277664.0,89.0,15236829.5,79.0,12.74,0.022472
2,1926-02-27,-0.033296,-0.037876,-0.039979,-0.042451,14712894.9,89.0,15277664.0,81.0,12.18,-0.043956
3,1926-03-31,-0.057708,-0.062007,-0.067915,-0.073275,14012079.2,89.0,14712894.9,81.0,11.46,-0.059113
4,1926-04-30,0.038522,0.034856,0.031441,0.027121,14500482.2,89.0,14012079.2,82.0,11.72,0.022688


In [12]:
# TEST CODE
# Perform sentiment analysis on filing documents
import pysentiment2 as ps

# get the Harvard general sentiment dictionary
hiv4 = ps.HIV4()

# get the Loughran and McDonald dictionary
lm = ps.LM()

In [25]:
# TEST CODE
# get the sentiment score for each filing document

# calculate the sentiment score using Harvard dictionary
hiv4_score = hiv4.get_score(filtered_corpus)
print("Harvard dictionary score: " + str(hiv4_score))

# calculate the sentiment score using Loughran dictionary
lm_score = lm.get_score(filtered_corpus)
print("Loughran dictionary score: " + str(lm_score))

Harvard dictionary score: {'Positive': 1979, 'Negative': 882, 'Polarity': 0.3834323661714672, 'Subjectivity': 0.04226121894232827}
Loughran dictionary score: {'Positive': 226, 'Negative': 366, 'Polarity': -0.23648648608701608, 'Subjectivity': 0.0087447191939386}


In [ ]:
# Perform sentiment analysis on filing documents
import pysentiment2 as ps

# this function calculates the sentiment score of the text using a dictionary
# (Loughran or Harvard) passed in as a parameter
def get_sentiment_score(text, dictionary):
    # get the Harvard general sentiment dictionary
    hiv4 = ps.HIV4()
    # get the Loughran and McDonald dictionary
    lm = ps.LM()

    if dictionary == "Harvard":
        return hiv4.get_sentiment_score(text)

    if dictionary == "Loughran":
        return lm.get_sentiment_score(text)

    raise ValueError("The dictionary could not be found.")

In [36]:
# TEST CODE
# load CRSP data for calculating Excess return

CRSP_data_dir = "CRSP-data/"
crsp_df = pd.read_csv(CRSP_data_dir + "crsp.csv", parse_dates=['date'])
print(crsp_df.head())

/var/folders/4l/kfc4gh5d1jn6zkrmv51vvbyw0000gn/T/ipykernel_2735/2278340170.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_df = pd.read_csv(CRSP_data_dir + "crsp.csv", parse_dates=['date'])


   PERMNO       date        RET    vwretd
0   10026 2019-01-02  -0.024829  0.001796
1   10026 2019-01-03   0.014326 -0.021043
2   10026 2019-01-04   0.012725  0.033410
3   10026 2019-01-07   0.003935  0.009202
4   10026 2019-01-08   0.022626  0.010269


In [61]:
# TEST CODE
sp500_permno_nums = sp500_sample[['permno', 'ticker']].copy()
print(sp500_permno_nums.head())

         permno ticker
120131  34746.0   FITB
120132  75591.0    IEX
120133  14541.0    CVX
120134  14593.0   AAPL
120135  75341.0    DRE


In [60]:
# TEST CODE
print(type(crsp_df['PERMNO'][0]))
print(type(sp500_permno_nums['permno']))
print(sp500_permno_nums.iloc[0])

<class 'numpy.int64'>
<class 'pandas.core.series.Series'>
permno    34746.0
Name: 120131, dtype: float64


In [59]:
# this function retrieves Value-Weighted Return (including distributions) and
# Holding Period Return from the CRSP database
def get_crsp_data():
    CRSP_DATA_DIR = "CRSP-data/"
    crsp_df = pd.read_csv(CRSP_DATA_DIR + "crsp.csv", parse_dates=['date'])
    return crsp_df